# Custom TensorFlow estimator

The objective of this notebook is to show you how one can implement a custom TF estimator, the PCA is only an illustrative application here. 

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.ERROR)
tf.enable_eager_execution()
tf.__version__

## Random data

We generate some random data, feel free use real data if you like (such as stock prices). We center the data.

In [ ]:
nz = 3 # latent factor dim
nx = 10 # features dim
n = 1000 # number of observations
seed = 1 

## generate data
np.random.seed(seed)
z_sample = np.random.randn(n, nz)
B_mat = np.random.randn(nx, nz)
x_sample = (B_mat @ z_sample.T).T

## center data
x_sample = x_sample - x_sample.mean(axis=0, keepdims=True)

## PCA with SVD

We compute the exact solution using the SVD decomposition, see [here](https://en.wikipedia.org/wiki/Principal_component_analysis#Singular_value_decomposition).

In [ ]:
# SVD
ss, us, vs = tf.svd(x_sample, full_matrices=False, compute_uv=True)
# 1st eigenvector
w1_svd = vs[:,0].numpy()
print(w1_svd)

## Build custom estimator

We build a model to learn the first coordinate transformation (eigenvector).

In [ ]:
def model_fn(features, labels, mode, params):
    # input data
    #x = tf.feature_column.input_layer(features, params['feature_columns'])
    x = features["x"] 
    
    # number of features
    nx = params['n_features']
    l1p = params['l1_pen']
    lr = params['lr']
    
    # initial value for w1
    w_init = np.array([1.0/nx] * nx).astype(np.float32).reshape(-1,1)
    # 1st eigenvector
    w1 = tf.compat.v1.get_variable(name="w1", 
                                   initializer= w_init,
                                   constraint=lambda x: tf.clip_by_value(x, -1, 1))
       
    if mode == tf.estimator.ModeKeys.PREDICT:
        ## xxxx
        ## this part is just returning w1 in our simple example here
        ## BUT in regression/classification it should return the predictions for 'y'
        ## xxxx
        predictions = {'w1' : tf.identity(w1)}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # new coordinates of x
    z = tf.matmul(x, w1)

    # objective = max variance of projected points
    l0 = -tf.reduce_mean(tf.pow(z,2))
    # regularization = L2 norm of w1 should be equal to one
    l1 = l1p * tf.pow(tf.reduce_sum(tf.pow(w1,2)) - 1, 2)
    # loss function
    loss = l0 + l1
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdagradOptimizer(learning_rate=lr)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


## Setup and train the estimator

In [ ]:
# hyper-params
n_step = 500
lr = 0.1
lambda_1 = 1000.0 ## big penalty needed!

# simple data ingestion
input_fn = lambda: ({"x":tf.convert_to_tensor(x_sample, dtype=tf.float32)}, {})

# make TFest
my_est = tf.estimator.Estimator(
    model_fn=model_fn,
    params={'n_features' : nx, 
            'l1_pen' : lambda_1,
            'lr' : lr
            }
    )

## train
my_est.train(input_fn, steps=n_step)

preds = my_est.predict(input_fn=lambda: ({"x": None}, {}),
                       yield_single_examples=False) ## ouput w1 so no need for any input
w1_tfe = next(preds)['w1'].reshape(-1)
print(w1_tfe)

## Compare results

In [ ]:
def print_summary(w, name):
    ## display basic info about w
    print('*' * 5, '\n', name, ":\n",
          '\tw1 norm = ', np.sum(w**2), 
          '\n\tweights =', w,
          '\n\tvar x proj =', np.sum((x_sample @ w)**2)/n**2)

print_summary(w1_tfe, 'PCA TFE')    
print_summary(w1_svd, 'PCA SVD')

# Sign of axes is arbitrary !!!
if np.sign(w1_svd[0]) != np.sign(w1_tfe[0]):
    w1_tfe = - w1_tfe

x = np.arange(nx)  # the label locations
width = 0.45  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, w1_tfe, width, label='TFE')
rects2 = ax.bar(x + width/2, w1_svd, width, label='SVD')
ax.set_xlabel('coordinate')
ax.set_ylabel('w1 values')
ax.legend()
plt.show()

## Exercises

* return the principal components in addition to $w1$ in the prediction step.
* how would change the above to learn multiple eigenvectors?